## Fake News Classifier Using LSTM

In [83]:
import pandas as pd

In [84]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
df=pd.read_csv('/content/drive/MyDrive/fake-news/train.csv')
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [86]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [87]:
###Drop Nan Values
df=df.dropna()


In [88]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [89]:
## Get the Dependent features
y=df['label']

In [90]:
X.shape

(18285, 4)

In [91]:
y.shape

(18285,)

In [92]:
import tensorflow as tf

In [93]:
tf.__version__

'2.17.0'

In [94]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [95]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [96]:
messages=X.copy()

In [97]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [98]:
messages.reset_index(inplace=True)

In [99]:
import nltk
import re
from nltk.corpus import stopwords

In [100]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [101]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [102]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [103]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1664, 1893, 56, 225, 2486, 3410, 3940, 4678, 4750, 2467],
 [3682, 4136, 815, 3408, 1623, 2727, 999],
 [3892, 42, 3629, 2998],
 [1996, 1869, 4217, 2488, 3024, 2950],
 [3439, 1623, 2923, 1962, 575, 4939, 1623, 3352, 1176, 3111],
 [4287,
  4882,
  2180,
  2726,
  1306,
  3694,
  875,
  1786,
  2101,
  4582,
  1609,
  2291,
  3229,
  3105,
  999],
 [2043, 2832, 4224, 81, 2692, 2720, 2919, 1833, 137, 4275, 3327],
 [4712, 432, 1476, 1337, 3372, 413, 3694, 741, 137, 4275, 3327],
 [4276, 87, 986, 303, 1507, 1196, 4432, 2888, 3694, 196],
 [3655, 2202, 3055, 1007, 4126, 3474, 4093, 285],
 [2735, 1067, 4785, 1048, 3510, 2581, 3353, 1362, 4598, 2925, 3464],
 [2488, 2530, 2486, 1196, 3694, 3372],
 [673, 3543, 849, 3268, 2362, 632, 1026, 3275, 2981],
 [2168, 218, 1201, 2010, 1875, 3735, 4751, 137, 4275, 3327],
 [1112, 2870, 4861, 1848, 3986, 137, 4275, 3327],
 [4530, 383, 417, 1603, 2271, 242, 2174, 711, 392, 1248],
 [2180, 504, 4136],
 [1526, 3969, 4334, 3181, 3694, 1342, 1761, 999],
 [1101, 1378

### Embedding Representation

In [104]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4678 4750 2467]
 [   0    0    0 ... 1623 2727  999]
 [   0    0    0 ...   42 3629 2998]
 ...
 [   0    0    0 ...  137 4275 3327]
 [   0    0    0 ... 4051 3879 2660]
 [   0    0    0 ... 2367 3005  925]]


In [105]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1664,
       1893,   56,  225, 2486, 3410, 3940, 4678, 4750, 2467], dtype=int32)

In [106]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [107]:
len(embedded_docs),y.shape

(18285, (18285,))

In [108]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [109]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [111]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7825 - loss: 0.4242 - val_accuracy: 0.9118 - val_loss: 0.1968
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9383 - loss: 0.1509 - val_accuracy: 0.9185 - val_loss: 0.1967
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9617 - loss: 0.1033 - val_accuracy: 0.9130 - val_loss: 0.2281
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9787 - loss: 0.0668 - val_accuracy: 0.9026 - val_loss: 0.2468
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9827 - loss: 0.0530 - val_accuracy: 0.9137 - val_loss: 0.2621
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9893 - loss: 0.0352 - val_accuracy: 0.9047 - val_loss: 0.3210
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9942 - loss: 0.0228 - val_accuracy: 0.9145 - val_loss: 0.3547
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9961 - loss: 0.0143 - val_accuracy: 0.

### Adding Dropout

In [112]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [113]:
# Get predictions (returns probabilities)
y_pred = model.predict(X_test)

# Convert probabilities to class labels
y_pred = predictions.argmax(axis=1)


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [114]:
from sklearn.metrics import confusion_matrix

In [115]:
confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [116]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917